In [1]:
import numpy
import pandas as pd
import cloudstorage as gcs
import pickle
from pandas.io.json import json_normalize
import time
import os
from gcloud import storage

In [2]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'C:\\Users\\alanl\\Desktop\\Data Flow Test-45c6beb1f9c3.json'

client = storage.Client()
bucket = client.get_bucket('py_spark_ds')

blobs = bucket.list_blobs()

In [3]:
def json_norm(player_list,feature):
    """
    takes the final list containing dictionary and returns parsed json
    :param final_list:
    :param player_name:
    :return: pandas data frame
    """
    df = player_list[feature]
    df = json_normalize(df)
    df.index = df.name
    df = df.drop(['name'],axis=1)
    df_series = df.stack()
    df_normalized = df_series.to_frame().T
    df_normalized.columns = ['_'.join(col).strip() for col in df_normalized.columns.values]
    
    return df_normalized

In [4]:
def convert_int(dataframe):
    """
    convert objects to int
    :param dataframe: 
    :return: 
    """
    for col in dataframe.columns:
        try:
            dataframe[col] = dataframe[col].fillna('999.99')
            dataframe[col] = dataframe[col].astype(int)
        except:
            continue
    return dataframe.loc[:,dataframe.dtypes==int]

In [5]:
def fill_missing(dataframe):
    for col in dataframe.columns:
        dataframe[col] = dataframe[col].fillna(999.99)
    return dataframe

In [6]:
print('Blobs:')
player_info = pd.DataFrame([])
for file in blobs:
    s = bucket.get_blob(file.name)
    with open("./tmp.pkl", "wb") as file_obj:
        s.download_to_file(file_obj)
    try:
        with open("./tmp.pkl","rb") as file_obj:
            tmp = pickle.load(file_obj)
    except:
        continue
        
    weaponCategory = json_norm(tmp,'weaponCategory')
    players = pd.DataFrame([{keys:values for keys,values in tmp['player'].items() if keys in ['name']}])
    stats = json_normalize([{keys:values for (keys,values) in tmp['stats'].items() if keys not in ['modes']}])
    
    #clean up vehicleCategory to include in dataframe
    vehicleCategory = tmp['vehicleCategory']
    vehicleCategory = json_normalize(vehicleCategory)
    vehicleCategory= vehicleCategory[['extra.kpm','extra.spm','name','stat.destroys','stat.kills','stat.score','stat.time']]
    vehicleCategory.index = vehicleCategory.name
    vehicleCategory = vehicleCategory.drop(['name'],axis=1)
    vehicleCategory_series = vehicleCategory.stack()
    vehicleCategory_normalized = vehicleCategory_series.to_frame().T
    vehicleCategory_normalized.columns = ['_'.join(col).strip() for col in vehicleCategory_normalized.columns.values]    
    
    vehicleCategory = vehicleCategory_normalized
    
 
    # clean up modes to include in dataframe
    modes = tmp['stats']['modes']
    modes = json_normalize(modes)
    modes = modes.drop(['id'], axis=1)
    modes.index = modes.name
    modes = modes.drop(['name'],axis=1)
    modes_series = modes.stack()
    modes_normalized = modes_series.to_frame().T
    modes_normalized.columns = ['_'.join(col).strip() for col in modes_normalized.columns.values]
    
    modes = modes_normalized
    
    
    os.remove("./tmp.pkl") 

    stats = convert_int(stats)
    modes = convert_int(modes)
    weaponCategory = convert_int(weaponCategory)
    vehicleCategory = convert_int(vehicleCategory)
    
    stats = fill_missing(stats)
    modes = fill_missing(modes)
    weaponCategory = fill_missing(weaponCategory)
    vehicleCategory = fill_missing(vehicleCategory)
    

    s = pd.concat([players,stats,modes, weaponCategory, vehicleCategory],axis=1)
    try:
        player_info = player_info.append(s,ignore_index=True)
        print("Success")
        print(player_info.shape)
    except Exception as e:
        print(e.message)
        continue

Blobs:
Success
(1, 394)
Success
(2, 394)
Success
(3, 394)
Success
(4, 394)
Success
(5, 394)
Success
(6, 394)
Success
(7, 394)
Success
(8, 394)
Success
(9, 394)
Success
(10, 394)
Success
(11, 394)
Success
(12, 394)
Success
(13, 394)
Success
(14, 394)
Success
(15, 394)
Success
(16, 394)
Success
(17, 394)
Success
(18, 394)
Success
(19, 394)
Success
(20, 394)
Success
(21, 394)
Success
(22, 394)
Success
(23, 394)
Success
(24, 394)
Success
(25, 394)
Success
(26, 394)
Success
(27, 394)
Success
(28, 394)
Success
(29, 394)
Success
(30, 394)
Success
(31, 394)
Success
(32, 394)
Success
(33, 394)
Success
(34, 394)
Success
(35, 394)
Success
(36, 394)
Success
(37, 394)
Success
(38, 394)
Success
(39, 394)
Success
(40, 394)
Success
(41, 394)
Success
(42, 394)
Success
(43, 394)
Success
(44, 394)
Success
(45, 394)
Success
(46, 394)
Success
(47, 394)
Success
(48, 394)


In [7]:
player_info.head()

name  avengerKills  bestStreak  deaths  dogtagsTaken  elo  \
0  AKATH3SCI3NTIST         41635          11  246916          3484    0   
1     Adversity 64         33602           3  182750          1393    0   
2   BOOTYxKILLERv1         23983           3  107761          3159    0   
3         Benz0 92         19239           4  174051          1352    0   
4  Bringmeofasking         33719           2  166132          3900    0   

   extra.accuracy  extra.assignments  extra.assignmentsTotal  extra.gspm  ...  \
0              13                 98                     108         255  ...   
1              19                 85                     108         227  ...   
2              18                 96                     108         152  ...   
3              12                 99                     108         219  ...   
4              24                 90                     108         206  ...   

   AA 9K22 Tunguska_extra.spm  AA 9K22 Tunguska_stat.destroys  \
0                           0                               0   
1                           0                               0   
2                           0                               0   
3                           0                               0   
4                           0                               0   

   AA 9K22 Tunguska_stat.kills  AA 9K22 Tunguska_stat.time  \
0                            0                           0   
1                            0                           0   
2                            0                           0   
3                            0                           0   
4                            0                           0   

   AA LAV-AD_extra.kpm  AA LAV-AD_extra.spm  AA LAV-AD_stat.destroys  \
0                    0                    0                        0   
1                    0                    0                        0   
2                    0                    0                        0   
3                    0                    0                        0   
4                    0                    0                        0   

   AA LAV-AD_stat.kills  AA LAV-AD_stat.score  AA LAV-AD_stat.time  
0                     0               3582630                    0  
1                     0                744924                    0  
2                     0               5793980                    0  
3                     0               3883420                    0  
4                     0               1049280                    0  

[5 rows x 394 columns]

In [9]:
player_info.to_csv('player_info.csv')

In [11]:
# upload the data into google cloud as csv, you'll find the file in the original bucket, then ssh
# into pyspark to process that data
player_info.to_csv('player_info.csv',index=True)
blob = bucket.blob('player_info.csv')
blob.upload_from_filename('C:\\Users\\alanl\\Desktop\\Repository\\Data_Science_Bootcamp_2019\\player_info.csv')
print('File has been uploaded to google_cloud')

File has been uploaded to google_cloud
